In [4]:
# import numpy as sns
# import pandas as np
# import seaborn as pd
# import matplotlib.pyplot as torch
# import catboost as plt

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

from lazypredict.Supervised import LazyRegressor
from catboost import CatBoostRegressor
from sklearn.impute import KNNImputer
from sklearn.ensemble import ExtraTreesRegressor, VotingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split, LearningCurveDisplay, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchsummary import summary
from torch.nn import functional as F
 
device = torch.device('cpu')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme()

from copy import deepcopy
import pickle

RANDOM_STATE = 42

# Как предсказать ZOI_drug_NP
 - Выполнить ```get_prediction(show_features=True)```. Получится список фич, которые должны быть в данных.
 - Закодировать значения категориальных фичей (вручную). Правила ниже

In [29]:
import glob
lst = glob.glob('a*.pkl')
lst

['a24.pkl',
 'a18.pkl',
 'a.pkl',
 'a1.pkl',
 'a2.pkl',
 'a19.pkl',
 'a14.pkl',
 'a16.pkl',
 'a13.pkl',
 'a6.pkl',
 'a3.pkl',
 'a0.pkl',
 'a7.pkl',
 'a15.pkl',
 'a17.pkl',
 'a20.pkl']

In [30]:
for file in lst:
    with open(file, 'rb') as a:
        cats = pickle.load(a)
    print(cats)

{
    "0": "blood",
    "1": "food",
    "2": "lungs",
    "3": "mammary gland",
    "4": "meat",
    "5": "mouth",
    "6": "nose",
    "7": "respiratory tract",
    "8": "skin",
    "9": "soil",
    "10": "sputum",
    "11": "urine",
    "12": "water"
}
{
    "0": "Acinetobacter",
    "1": "Actinobacillus",
    "2": "Bacillus",
    "3": "Candida",
    "4": "Candida glaebosa",
    "5": "Enterobacter",
    "6": "Enterococcus",
    "7": "Escherichia",
    "8": "Haemophilus",
    "9": "Klebsiella",
    "10": "Listeria",
    "11": "Micrococcus",
    "12": "Nakaseomyces",
    "13": "Neisseria",
    "14": "Pasteurella",
    "15": "Proteus",
    "16": "Pseudomonas",
    "17": "Salmonella",
    "18": "Serratia",
    "19": "Staphylococcus",
    "20": "Streptococcus",
    "21": "Vibrio"
}
{
    "0": "blood",
    "1": "food",
    "2": "lungs",
    "3": "mammary gland",
    "4": "meat",
    "5": "mouth",
    "6": "nose",
    "7": "respiratory tract",
    "8": "skin",
    "9": "soil",
    "10": "s

In [5]:
def get_prediction(df: pd.DataFrame or None=None, choosen_model='ExtraTreesRegressor', show_features=False): # or CatBoostRegressor or VotingRegressor
    '''
    Automatically make prediction, including encoding and everythin

    Parameters
    -----------
    df: DataFrame
        Dataset to prediction must contains same features
        (use show_features)
    choosen_model: str default="ExtraTreesRegressor"
        Use model from fitted_models.pkl, available models:
        ExtraTreesRegressor, CatBoostRegressor, VotingRegressor 
    show_features: bool default=False
        Print necessary features
    
    Returns
    -------
    predicted_values
    '''

    fitted_models = pickle.load(open('fitted_models.pkl', 'rb'))
    features = fitted_models['features']

    if show_features:
        print(features)
        return
    
    model = fitted_models[choosen_model]
    
    df_codes = deepcopy(df[features])

    for col in df[features]:
        if df[col].dtype == object:
            df_codes[col] = pd.Categorical(df[col])
            df_codes[col] = df_codes[col].cat.codes
            
    return model.predict(df_codes)


In [6]:
get_prediction(show_features=True)

Index(['growth_temp, C', 'LabuteASA', 'MolLogP',
       'fold_increase_in_antibacterial_activity (%)', 'avg_Incub_period',
       'method', 'Drug_class_drug_bank', 'Drug', 'NumAliphaticRings', 'shape',
       'Chi0', 'class', 'order', 'MDR_check', 'MolWt', 'NumHDonors',
       'ZOI_drug', 'avg_NP_size', 'phylum', 'family', 'lg_Drug_dose',
       'species', 'NP_Synthesis', 'genus', 'isolated_from', 'MinPartialCharge',
       'Bacteria', 'MaxPartialCharge', 'NumHAcceptors'],
      dtype='object')
